<a href="https://colab.research.google.com/github/thongchaiphumphuang/SGA_Adhoc/blob/main/CBD_create_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gspread==5.4.0  # this version has '.copy_to()' method.

import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import sqlite3
import gspread
from datetime import date, datetime, timedelta
import re

from oauth2client.client import GoogleCredentials
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds) # authorize Google Colab using spreadsheet.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

# import gspread
# from oauth2client.client import GoogleCredentials

# gc = gspread.authorize(GoogleCredentials.get_application_default())

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Mounted at /content/drive


In [4]:
def read_data(key, sheet_name):
    wb = gc.open_by_key(key)
    sh = wb.worksheet(sheet_name)
    data = sh.get_all_values()
    df = pd.DataFrame.from_records(data[1:], columns=data[0])
    return df

In [5]:
key_google_sheet = '1qz1zBHx9MRxkbncDR5xpmI6WG-0Cw7joci1TGgE_gUU'
s = gc.open_by_key(key_google_sheet)
worksheet_list  = s.worksheets()

In [6]:
worksheet_list

[<Worksheet 'Database' id:0>,
 <Worksheet 'SI=SO' id:2100948695>,
 <Worksheet 'db-Subdist' id:1539514961>,
 <Worksheet 'check INV = 0_old' id:336376548>,
 <Worksheet 'check INV = 0' id:2103849276>,
 <Worksheet 'db-CP&A' id:1485527915>,
 <Worksheet 'Pivot Table 1' id:18027064>,
 <Worksheet 'Inventory end of month' id:918988076>,
 <Worksheet 'SI' id:2089943318>,
 <Worksheet 'SO' id:64488386>,
 <Worksheet 'Index_customer' id:39058727>,
 <Worksheet 'Pivot Table 2' id:148398656>,
 <Worksheet 'Sheet1' id:320466114>,
 <Worksheet 'Connected Sheet 1' id:602069246>,
 <Worksheet 'Extract 1' id:1690859388>]

#Read data

##Inventory end of month

In [203]:
sheet_name = 'Inventory end of month'
inv_df = read_data(key_google_sheet, sheet_name)
inv_df['Date'] = pd.to_datetime(inv_df['Date'])

inv_df['Ending Qty.'] = inv_df['Ending Qty.'].astype('float')
inv_df['Total Stock'] = inv_df['Total Stock'].astype('float')
inv_df['Adjustment'] = inv_df['Adjustment'].astype('float')
inv_df['Sale Return'] = inv_df['Sale Return'].astype('float')
adj_df = inv_df.groupby(['Date', 'PPGs', 'Item Code'], as_index=False)[['Adjustment', 'Sale Return']].sum()
adj_df.columns = ["Date", "PPG", "SKU", "Adj", "Return"]
inv_df = inv_df.groupby(['Date', 'PPGs', 'Item Code'], as_index = False)[['Total Stock']].sum()
inv_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_df.head()

,Date,PPG,SKU,INV
0,2021-12-01,,61002888,1756.00
1,2021-12-01,,TH03447A,94.00
2,2021-12-01,,TH03448A,109.00
3,2021-12-01,,TH03449A,112.00
4,2021-12-01,#N/A,1051531,81.56


In [204]:
adj_df.head()

,Date,PPG,SKU,Adj,Return
0,2021-12-01,,61002888,0.0,0.0
1,2021-12-01,,TH03447A,0.0,0.0
2,2021-12-01,,TH03448A,0.0,0.0
3,2021-12-01,,TH03449A,0.0,0.0
4,2021-12-01,#N/A,1051531,0.0,0.0


In [ ]:
inv_jan_df = inv_df[inv_df['Date'] == '2021-12-01']
inv_jan_df

,Date,PPG,SKU,INV
0,2021-12-01,,61002888,1756.00
1,2021-12-01,,TH03447A,94.00
2,2021-12-01,,TH03448A,109.00
3,2021-12-01,,TH03449A,112.00
4,2021-12-01,#N/A,1051531,81.56
...,...,...,...,...
138,2021-12-01,Total 110 / 150 g Single,61006655,424.06
139,2021-12-01,Total 110 / 150 g Single,61012801,480.85
140,2021-12-01,Total 110 / 150 g Single,TH02939A,1.25
141,2021-12-01,Total 110 / 150 g Single,TH02940A,3.52


In [ ]:
inv_jan_df['Date'] = inv_jan_df['Date']+pd.DateOffset(months=1)

<ipython-input-54-48c0e7ccf608>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_jan_df['Date'] = inv_jan_df['Date']+pd.DateOffset(months=1)


In [ ]:
inv_jan_df

,Date,PPG,SKU,INV
0,2022-01-01,,61002888,1756.00
1,2022-01-01,,TH03447A,94.00
2,2022-01-01,,TH03448A,109.00
3,2022-01-01,,TH03449A,112.00
4,2022-01-01,#N/A,1051531,81.56
...,...,...,...,...
138,2022-01-01,Total 110 / 150 g Single,61006655,424.06
139,2022-01-01,Total 110 / 150 g Single,61012801,480.85
140,2022-01-01,Total 110 / 150 g Single,TH02939A,1.25
141,2022-01-01,Total 110 / 150 g Single,TH02940A,3.52


In [ ]:
adj_jan_df = adj_df[adj_df['Date']  == "2022-01-01"]
adj_feb_df = adj_df[adj_df['Date']  == "2022-02-01"]
adj_mar_df = adj_df[adj_df['Date']  == "2022-03-01"]
adj_apr_df = adj_df[adj_df['Date']  == "2022-04-01"]
adj_may_df = adj_df[adj_df['Date']  == "2022-05-01"]
adj_jun_df = adj_df[adj_df['Date']  == "2022-06-01"]
adj_jul_df = adj_df[adj_df['Date']  == "2022-07-01"]
adj_aug_df = adj_df[adj_df['Date']  == "2022-08-01"]
adj_sep_df = adj_df[adj_df['Date']  == "2022-09-01"]
adj_oct_df = adj_df[adj_df['Date']  == "2022-10-01"]
adj_nov_df = adj_df[adj_df['Date']  == "2022-11-01"]

##Sale-In

In [ ]:
sheet_name = 'SI'
si_df = read_data(key_google_sheet, sheet_name)
si_df = si_df[["Date_time", "PPG", "SKU No.", "SKU Description", "Unit (CTN)"]]
si_df['Date_time'] = pd.to_datetime(si_df['Date_time'])
si_df['Unit (CTN)'] = si_df['Unit (CTN)'].astype('float')
si_df = si_df.groupby(['Date_time', 'PPG','SKU No.'], as_index=False)[['Unit (CTN)']].sum()
si_df.columns = ['Date', 'PPG','SKU','SI']
si_df

,Date,PPG,SKU,SI
0,2022-01-01,,61022100,20.0
1,2022-01-01,,TH03447A,300.0
2,2022-01-01,,TH03448A,150.0
3,2022-01-01,,TH03449A,160.0
4,2022-01-01,#N/A,PZM00205,647.0
...,...,...,...,...
553,2022-11-01,Plax 500ml,61013259,50.0
554,2022-11-01,Salt 180g/160g/150g,61012835,5000.0
555,2022-11-01,Salt 180g/160g/150g,61020027,2650.0
556,2022-11-01,Salt 180g/160g/150g Twin,TH03491A,100.0


In [ ]:
si_jan_df = si_df[si_df['Date']  == "2022-01-01"]
si_feb_df = si_df[si_df['Date']  == "2022-02-01"]
si_mar_df = si_df[si_df['Date']  == "2022-03-01"]
si_apr_df = si_df[si_df['Date']  == "2022-04-01"]
si_may_df = si_df[si_df['Date']  == "2022-05-01"]
si_jun_df = si_df[si_df['Date']  == "2022-06-01"]
si_jul_df = si_df[si_df['Date']  == "2022-07-01"]
si_aug_df = si_df[si_df['Date']  == "2022-08-01"]
si_sep_df = si_df[si_df['Date']  == "2022-09-01"]
si_oct_df = si_df[si_df['Date']  == "2022-10-01"]
si_nov_df = si_df[si_df['Date']  == "2022-11-01"]


## Sale-Out

In [ ]:
sheet_name = 'SO'
so_df = read_data(key_google_sheet, sheet_name)
so_df = so_df[["Date_time", "Customer_map", "PPGs","Inventory ID", "Description", "Total Quantity", "Order Type"]]
so_df.columns = ["Date_time", "Customer_map", "PPG", "SKU No.", "SKU Description", "Total Quantity", "Order Type"]
so_df['Type'] = so_df['Order Type'].apply(lambda x: -1.0 if x == "CM" else 1.0)
so_df['Date_time']  =pd.to_datetime(so_df['Date_time'])
so_df['Total Quantity'] = so_df['Total Quantity'].str.replace(",", '')
so_df['Total Quantity'] = so_df['Total Quantity'].astype('float')
so_df['Total Quantity'] = so_df['Total Quantity'] * so_df['Type']

# so_df = so_df.groupby(['Date_time', 'SKU No.'], as_index=False)[['Total Quantity']].sum()
# so_df.columns = ['Date', 'SKU No.', 'SO']
so_df

,Date_time,Customer_map,PPG,SKU No.,SKU Description,Total Quantity,Order Type,Type
0,2022-01-01,Phnom Penh 2,PO Shower 500g,TH02856A,""" Palmolive "" Shower Gel Calming Pleasure 500 ml",-5.00,CM,-1.0
1,2022-01-01,Phnom Penh 2,PO Shower 500g,TH02855A,""" Palmolive"" Shower Gel Nourish Delight 500 ml",-6.00,CM,-1.0
2,2022-03-01,RTK+MDK+KRT+STR,Big Red 50g/40g/35g,1524934,"""Colgate"" Toothpaste Double Cool Stripe 35g En...",-3.40,CM,-1.0
3,2022-03-01,RTK+MDK+KRT+STR,Big Red 140/150/170/180g,TH03575A,"""Colgate""Toothpaste Double Cool Stripe 140g",-17.21,CM,-1.0
4,2022-03-01,RTK+MDK+KRT+STR,Salt 35 / 40 g,TH03485A,"""Colgate"" Toothpaste Salt Herbal Fluoride 35g new",-0.17,CM,-1.0
...,...,...,...,...,...,...,...,...
16670,2022-11-01,Phnom Penh 2,CTB SS S.Premium P5,61007737,"""Colgate"" Toothbrush Slim Soft Charcoal Pack5",50.00,IN,1.0
16671,2022-11-01,Phnom Penh 2,CTB Value Kids,1051826,"""Colgate"" Toothbrush Value Kids",20.00,IN,1.0
16672,2022-11-01,Phnom Penh 2,PALMOLIVE SHAMPOO 6G,1529601,"""Palmolive""Shampoo Intensive Moisture Pink 6g",10.00,IN,1.0
16673,2022-11-01,Phnom Penh 2,,TH03447A,"""Palmolive""Shampoo& Conditioner Intensive Mois...",20.00,IN,1.0


In [ ]:
so_df = so_df.groupby(['Date_time', 'PPG','SKU No.'], as_index=False)[['Total Quantity']].sum()
so_df.columns = ["Date", "PPG", "SKU", "SO"]

In [ ]:
so_jan_df  = so_df[so_df['Date']  == '2022-01-01']
so_feb_df  = so_df[so_df['Date']  == '2022-02-01']
so_mar_df  = so_df[so_df['Date']  == '2022-03-01']
so_apr_df  = so_df[so_df['Date']  == '2022-04-01']
so_may_df  = so_df[so_df['Date']  == '2022-05-01']
so_jun_df  = so_df[so_df['Date']  == '2022-06-01']
so_jul_df  = so_df[so_df['Date']  == '2022-07-01']
so_aug_df  = so_df[so_df['Date']  == '2022-08-01']
so_sep_df  = so_df[so_df['Date']  == '2022-09-01']
so_oct_df  = so_df[so_df['Date']  == '2022-10-01']
so_nov_df  = so_df[so_df['Date']  == '2022-11-01']

so_feb_df

,Date,PPG,SKU,SO
104,2022-02-01,,61002888,650.00
105,2022-02-01,,TH03447A,82.50
106,2022-02-01,,TH03448A,74.50
107,2022-02-01,,TH03449A,67.00
108,2022-02-01,Big Red 140/150/170/180g,61007290,21.00
...,...,...,...,...
203,2022-02-01,Total 110 / 150 g Single,61006652,1.00
204,2022-02-01,Total 110 / 150 g Single,61006655,33.00
205,2022-02-01,Total 110 / 150 g Single,61012801,161.13
206,2022-02-01,Total 110 / 150 g Single,TH02940A,0.50


In [ ]:
so_df.columns

Index(['Date', 'PPG', 'SKU', 'SO'], dtype='object')

In [ ]:
si_df.columns

Index(['Date', 'PPG', 'SKU', 'SI'], dtype='object')

In [ ]:
inv_df.columns

Index(['Date', 'PPG', 'SKU', 'INV'], dtype='object')

In [ ]:
merged_df = so_jan_df.merge(si_jan_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_jan_df, on =  ["Date", 'PPG', 'SKU'], how='outer')

In [ ]:
adj_jan_df

,Date,PPG,SKU,Adj,Return
143,2022-01-01,,61002888,0.0,0.0
144,2022-01-01,,61022100,0.0,0.0
145,2022-01-01,,TH03447A,0.0,0.0
146,2022-01-01,,TH03448A,0.0,0.0
147,2022-01-01,,TH03449A,0.0,0.0
...,...,...,...,...,...
280,2022-01-01,Total 110 / 150 g Single,61006655,0.0,0.0
281,2022-01-01,Total 110 / 150 g Single,61012801,0.0,0.0
282,2022-01-01,Total 110 / 150 g Single,TH02939A,0.0,0.0
283,2022-01-01,Total 110 / 150 g Single,TH02940A,0.0,0.0


In [ ]:
merged_df = merged_df.merge(adj_jan_df,on = ["Date", "PPG", "SKU"], how='left' )

In [ ]:
merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)

merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']

In [ ]:
jan_df = merged_df.copy()
jan_df

,Date,PPG,SKU,SO,SI,INV_PP,Adj,Return,INV_CP
0,2022-01-01,,61002888,335.00,0.0,1756.00,0.0,0.0,1421.00
1,2022-01-01,,TH03447A,223.50,300.0,94.00,0.0,0.0,170.50
2,2022-01-01,,TH03448A,211.50,150.0,109.00,0.0,0.0,47.50
3,2022-01-01,,TH03449A,204.50,160.0,112.00,0.0,0.0,67.50
4,2022-01-01,#N/A,1051531,3.89,0.0,81.56,0.0,0.0,77.67
...,...,...,...,...,...,...,...,...,...
148,2022-01-01,Sensitive 120g/110g/100g,1537256,0.00,0.0,0.00,0.0,0.0,0.00
149,2022-01-01,Total 110 / 150 g Single,61006652,0.00,0.0,2.29,0.0,0.0,2.29
150,2022-01-01,Total 110 / 150 g Single,TH02939A,0.00,0.0,1.25,0.0,0.0,1.25
151,2022-01-01,Total 110 / 150 g Single,TH02940A,0.00,0.0,3.52,0.0,0.0,3.52


In [ ]:
inv_feb_df = jan_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_feb_df

,Date,PPG,SKU,INV_CP
0,2022-01-01,,61002888,1421.00
1,2022-01-01,,TH03447A,170.50
2,2022-01-01,,TH03448A,47.50
3,2022-01-01,,TH03449A,67.50
4,2022-01-01,#N/A,1051531,77.67
...,...,...,...,...
148,2022-01-01,Sensitive 120g/110g/100g,1537256,0.00
149,2022-01-01,Total 110 / 150 g Single,61006652,2.29
150,2022-01-01,Total 110 / 150 g Single,TH02939A,1.25
151,2022-01-01,Total 110 / 150 g Single,TH02940A,3.52


In [ ]:
inv_feb_df['Date'] = inv_feb_df['Date']+pd.DateOffset(months=1)

<ipython-input-112-2cadfc806cc1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_feb_df['Date'] = inv_feb_df['Date']+pd.DateOffset(months=1)


In [ ]:
inv_feb_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_feb_df

,Date,PPG,SKU,INV
0,2022-02-01,,61002888,1421.00
1,2022-02-01,,TH03447A,170.50
2,2022-02-01,,TH03448A,47.50
3,2022-02-01,,TH03449A,67.50
4,2022-02-01,#N/A,1051531,77.67
...,...,...,...,...
148,2022-02-01,Sensitive 120g/110g/100g,1537256,0.00
149,2022-02-01,Total 110 / 150 g Single,61006652,2.29
150,2022-02-01,Total 110 / 150 g Single,TH02939A,1.25
151,2022-02-01,Total 110 / 150 g Single,TH02940A,3.52


In [ ]:
merged_df = so_feb_df.merge(si_feb_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_feb_df, on =  ["Date", 'PPG', 'SKU'], how='outer')
merged_df = merged_df.merge(adj_feb_df,on = ["Date", "PPG", "SKU"], how='left' )

merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']
# merged_df[['SO', 'SI',  'INV', "Adj",	"Return"]] = merged_df[['SO', 'SI', 'INV',"Adj",	"Return"]].fillna(0)
# merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP']

In [ ]:
merged_df

,Date,PPG,SKU,SO,SI,INV_PP,Adj,Return,INV_CP
0,2022-02-01,,61002888,650.0,640.0,1421.00,0.0,0.0,1411.00
1,2022-02-01,,TH03447A,82.5,150.0,170.50,0.0,0.0,238.00
2,2022-02-01,,TH03448A,74.5,100.0,47.50,0.0,0.0,73.00
3,2022-02-01,,TH03449A,67.0,200.0,67.50,0.0,0.0,200.50
4,2022-02-01,Big Red 140/150/170/180g,61007290,21.0,50.0,40.31,0.0,0.0,69.31
...,...,...,...,...,...,...,...,...,...
149,2022-02-01,PTX SC 180ml Bottle,TH02160A,0.0,0.0,0.00,0.0,0.0,0.00
150,2022-02-01,PTX SC 450/500ml (new pump),TH03474A,0.0,0.0,1.00,0.0,0.0,1.00
151,2022-02-01,Plax 500ml,1525673,0.0,0.0,0.00,0.0,0.0,0.00
152,2022-02-01,Sensitive 120g/110g/100g,1537256,0.0,0.0,0.00,0.0,0.0,0.00


In [ ]:
feb_df = merged_df.copy()

In [ ]:
inv_mar_df = feb_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_mar_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_mar_df['Date'] = inv_mar_df['Date']+pd.DateOffset(months=1)
merged_df = so_mar_df.merge(si_mar_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_mar_df, on =  ["Date", 'PPG', 'SKU'], how='outer')

merged_df = merged_df.merge(adj_mar_df,on = ["Date", "PPG", "SKU"], how='left' )

merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']


<ipython-input-117-9ede418606b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_mar_df['Date'] = inv_mar_df['Date']+pd.DateOffset(months=1)


In [ ]:
merged_df

,Date,PPG,SKU,SO,SI,INV_PP,Adj,Return,INV_CP
0,2022-03-01,,61002888,310.0,400.0,1411.00,0.0,0.0,1501.00
1,2022-03-01,,TH03447A,34.5,100.0,238.00,0.0,0.0,303.50
2,2022-03-01,,TH03448A,24.5,75.0,73.00,0.0,0.0,123.50
3,2022-03-01,,TH03449A,10.5,75.0,200.50,0.0,0.0,265.00
4,2022-03-01,Big Red 140/150/170/180g,61007290,83.0,20.0,69.31,0.0,0.0,6.31
...,...,...,...,...,...,...,...,...,...
152,2022-03-01,PTX SC 180ml Bottle,1506717,0.0,0.0,2.25,0.0,0.0,2.25
153,2022-03-01,PTX SC 180ml Bottle,TH02160A,0.0,0.0,0.00,0.0,0.0,0.00
154,2022-03-01,Plax 500ml,1525673,0.0,0.0,0.00,0.0,0.0,0.00
155,2022-03-01,Sensitive 120g/110g/100g,1537256,0.0,0.0,0.00,0.0,0.0,0.00


In [ ]:
mar_df = merged_df.copy()

In [ ]:
inv_apr_df = mar_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_apr_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_apr_df['Date'] = inv_apr_df['Date']+pd.DateOffset(months=1)
merged_df = so_apr_df.merge(si_apr_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_apr_df, on =  ["Date", 'PPG', 'SKU'], how='outer')
merged_df = merged_df.merge(adj_apr_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)

merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']


<ipython-input-120-7d6fa327cd8d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_apr_df['Date'] = inv_apr_df['Date']+pd.DateOffset(months=1)


In [ ]:
apr_df = merged_df.copy()

In [ ]:
inv_may_df = apr_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_may_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_may_df['Date'] = inv_may_df['Date']+pd.DateOffset(months=1)
merged_df = so_may_df.merge(si_may_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_may_df, on =  ["Date", 'PPG', 'SKU'], how='outer')
merged_df = merged_df.merge(adj_may_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']

<ipython-input-122-fc44b4a8f0eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_may_df['Date'] = inv_may_df['Date']+pd.DateOffset(months=1)


In [ ]:
may_df = merged_df.copy()
may_df

,Date,PPG,SKU,SO,SI,INV_PP,Adj,Return,INV_CP
0,2022-05-01,,61002888,479.35,0.0,1636.00,0.0,0.67,1157.32
1,2022-05-01,,TH03447A,171.17,0.0,249.50,0.0,0.00,78.33
2,2022-05-01,,TH03448A,32.17,0.0,165.50,0.0,3.00,136.33
3,2022-05-01,,TH03449A,37.17,0.0,247.00,0.0,3.00,212.83
4,2022-05-01,#N/A,1051531,14.72,0.0,77.67,0.0,0.00,62.95
...,...,...,...,...,...,...,...,...,...
153,2022-05-01,PTX SC 180ml Bottle,1506716,0.00,0.0,3.00,0.0,0.00,3.00
154,2022-05-01,PTX SC 180ml Bottle,1506717,0.00,0.0,2.25,0.0,0.00,2.25
155,2022-05-01,PTX SC 180ml Bottle,TH02160A,0.00,0.0,0.00,0.0,0.00,0.00
156,2022-05-01,Plax 500ml,1525673,0.00,0.0,0.00,0.0,0.00,0.00


In [ ]:
inv_jun_df = may_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_jun_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_jun_df['Date'] = inv_jun_df['Date']+pd.DateOffset(months=1)
merged_df = so_jun_df.merge(si_jun_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_jun_df, on =  ["Date", 'PPG', 'SKU'], how='outer')


merged_df = merged_df.merge(adj_jun_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']

<ipython-input-124-a50afafc0793>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_jun_df['Date'] = inv_jun_df['Date']+pd.DateOffset(months=1)


In [ ]:
jun_df = merged_df.copy()

In [ ]:
inv_jul_df = jun_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_jul_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_jul_df['Date'] = inv_jul_df['Date']+pd.DateOffset(months=1)
merged_df = so_jul_df.merge(si_jul_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_jul_df, on =  ["Date", 'PPG', 'SKU'], how='outer')
merged_df = merged_df.merge(adj_jul_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']

<ipython-input-125-ee4f7806f19a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_jul_df['Date'] = inv_jul_df['Date']+pd.DateOffset(months=1)


In [ ]:
jul_df = merged_df.copy()

In [ ]:
inv_aug_df = jul_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_aug_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_aug_df['Date'] = inv_aug_df['Date']+pd.DateOffset(months=1)
merged_df = so_aug_df.merge(si_aug_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_aug_df, on =  ["Date", 'PPG', 'SKU'], how='outer')

merged_df = merged_df.merge(adj_aug_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']
aug_df = merged_df.copy()

<ipython-input-127-c16651f25143>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_aug_df['Date'] = inv_aug_df['Date']+pd.DateOffset(months=1)


In [ ]:
inv_sep_df = aug_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_sep_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_sep_df['Date'] = inv_sep_df['Date']+pd.DateOffset(months=1)
merged_df = so_sep_df.merge(si_sep_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_sep_df, on =  ["Date", 'PPG', 'SKU'], how='outer')

merged_df = merged_df.merge(adj_sep_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']
sep_df = merged_df.copy()

<ipython-input-128-e8508a32c89a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_sep_df['Date'] = inv_sep_df['Date']+pd.DateOffset(months=1)


In [ ]:
inv_oct_df = sep_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_oct_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_oct_df['Date'] = inv_oct_df['Date']+pd.DateOffset(months=1)
merged_df = so_oct_df.merge(si_oct_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_oct_df, on =  ["Date", 'PPG', 'SKU'], how='outer')

merged_df = merged_df.merge(adj_oct_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)


merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']

oct_df = merged_df.copy()

<ipython-input-129-646f106119c0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_oct_df['Date'] = inv_oct_df['Date']+pd.DateOffset(months=1)


In [ ]:
inv_nov_df = oct_df[['Date', 'PPG', 'SKU', 'INV_CP']]
inv_nov_df.columns = ['Date', 'PPG', 'SKU', 'INV']
inv_nov_df['Date'] = inv_nov_df['Date']+pd.DateOffset(months=1)
merged_df = so_nov_df.merge(si_nov_df, on = ["Date", 'PPG' ,'SKU'], how='outer').merge(inv_nov_df, on =  ["Date", 'PPG', 'SKU'], how='outer')

merged_df = merged_df.merge(adj_nov_df,on = ["Date", "PPG", "SKU"], how='left' )


merged_df[['SO', 'SI',  'INV', 'Adj', 'Return']] = merged_df[['SO', 'SI', 'INV','Adj', 'Return']].fillna(0)

merged_df = merged_df[["Date", 'PPG', 'SKU','SO', 'SI',  'INV','Adj', 'Return']]
merged_df.columns = ["Date", 'PPG', 'SKU', 'SO', 'SI', 'INV_PP', 'Adj', 'Return']
merged_df['INV_CP'] = merged_df['INV_PP'] +merged_df['SI'] - merged_df['SO']  +merged_df['Adj'] + merged_df['Return']
nov_df = merged_df.copy()

<ipython-input-130-1d53a6f92cae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_nov_df['Date'] = inv_nov_df['Date']+pd.DateOffset(months=1)


In [ ]:
df = pd.concat([jan_df, feb_df,mar_df,apr_df,may_df,jun_df,jul_df,aug_df,sep_df,oct_df,nov_df])

In [ ]:
df.groupby(['Date','PPG', "SKU"], as_index=False).sum()

,Date,PPG,SKU,SO,SI,INV_PP,Adj,Return,INV_CP
0,2022-01-01,,61002888,335.00,0.0,1756.00,0.0,0.0,1421.00
1,2022-01-01,,61022100,0.00,20.0,0.00,0.0,0.0,20.00
2,2022-01-01,,TH03447A,223.50,300.0,94.00,0.0,0.0,170.50
3,2022-01-01,,TH03448A,211.50,150.0,109.00,0.0,0.0,47.50
4,2022-01-01,,TH03449A,204.50,160.0,112.00,0.0,0.0,67.50
...,...,...,...,...,...,...,...,...,...
1803,2022-11-01,Total 110 / 150 g Single,61006655,0.00,0.0,2.00,0.0,0.0,2.00
1804,2022-11-01,Total 110 / 150 g Single,61012801,80.86,0.0,153.62,0.0,0.0,72.76
1805,2022-11-01,Total 110 / 150 g Single,TH02939A,0.00,0.0,2.33,0.0,0.0,2.33
1806,2022-11-01,Total 110 / 150 g Single,TH02940A,0.00,0.0,3.15,0.0,0.0,3.15


In [ ]:
df.to_excel('check_inv.xlsx', index=False)

In [ ]:
df

,Date,PPG,SKU,SO,SI,INV_PP,INV_CP
0,2022-01-01,,61002888,335.00,0.0,1756.00,1421.00
1,2022-01-01,,TH03447A,223.50,300.0,94.00,170.50
2,2022-01-01,,TH03448A,211.50,150.0,109.00,47.50
3,2022-01-01,,TH03449A,204.50,160.0,112.00,67.50
4,2022-01-01,#N/A,1051531,3.89,0.0,81.56,77.67
...,...,...,...,...,...,...,...
174,2022-11-01,PALMOLIVE SHAMPOO 6G,1529701,0.00,0.0,0.00,0.00
175,2022-11-01,PO Men Shampoo 180ml,1529748,0.00,0.0,0.00,0.00
176,2022-11-01,PTX SC 180ml Bottle,TH02160A,0.00,0.0,0.00,0.00
177,2022-11-01,Plax 500ml,1525673,0.00,0.0,0.00,0.00


In [ ]:
df['Diff'] = df['INV_PP'] + df['SI'] - df['SO'] - df['INV_CP']
df.round(1)


,Date,PPG,SKU,SO,SI,INV_PP,INV_CP,Diff
0,2022-01-01,,61002888,335.0,0.0,1756.0,1421.0,0.0
1,2022-01-01,,TH03447A,223.5,300.0,94.0,170.5,0.0
2,2022-01-01,,TH03448A,211.5,150.0,109.0,47.5,0.0
3,2022-01-01,,TH03449A,204.5,160.0,112.0,67.5,0.0
4,2022-01-01,#N/A,1051531,3.9,0.0,81.6,77.7,0.0
...,...,...,...,...,...,...,...,...
174,2022-11-01,PALMOLIVE SHAMPOO 6G,1529701,0.0,0.0,0.0,0.0,0.0
175,2022-11-01,PO Men Shampoo 180ml,1529748,0.0,0.0,0.0,0.0,0.0
176,2022-11-01,PTX SC 180ml Bottle,TH02160A,0.0,0.0,0.0,0.0,0.0
177,2022-11-01,Plax 500ml,1525673,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = df.sort_values(['Date'])

In [ ]:
(df[df["Diff"] == 0].size / df.size)*100

100.0

In [ ]:
df

,Date,PPG,SKU,SO,SI,INV_PP,INV_CP,Diff
0,2022-01-01,,61002888,335.00,0.0,1756.00,1421.00,0.0
98,2022-01-01,Salt 35 / 40 g,TH03485A,949.70,1200.0,3731.05,3981.35,0.0
99,2022-01-01,Salt Herbal 35g+TP,61016296,50.00,0.0,83.06,33.06,0.0
100,2022-01-01,Super Flexi,1053201,1.00,0.0,380.65,379.65,0.0
101,2022-01-01,TB SLIMSOFT SINGLE - 30k,61014072,5.33,50.0,672.39,717.06,0.0
...,...,...,...,...,...,...,...,...
61,2022-11-01,PLAX 100 ML,1525308,0.67,0.0,0.94,0.27,0.0
62,2022-11-01,PLAX 100 ML,61013218,1.67,50.0,1.80,50.13,0.0
63,2022-11-01,POAT SG 750 ML,1507334,49.00,0.0,59.67,10.67,0.0
65,2022-11-01,PTX SC 180ml Bottle,61001526,23.50,10.0,16.35,2.85,0.0


# Check SO = SI

In [205]:
sheet_name = 'db-Subdist'
sub_df = read_data(key_google_sheet, sheet_name)
sub_df.head()

,Date,Dist. Group,Distributor,Code,PPGs,DESCRIPTION PRODUCT,Packing,Price/Ctn,OP (VOL),SI (VOL),...,OP (VAL),SI (VAL),SI FOC (VAL),RI (VAL),RI FOC (VAL),RO (VAL),TOT (VAL),SO (VAL),SO FOC (VAL),INV (VAL)
0,1/31/2022,Xborder,Battambang,1523871,Big Red 25g/20g,"""Colgate"" Toothpaste Great Regular Flavor 20g.",288,45,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1/31/2022,Xborder,Battambang,61007304,Big Red 50g/40g/35g,"""Colgate"" Toothpaste Great Regular Flavor 35g",216,72,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1/31/2022,Xborder,Battambang,TH03541A,Big Red 50g/40g/35g,"""Colgate"" Toothpaste Great Regular Flavor 40g.",216,77,0,0,...,6,0,0,0,0,0,6,0,0,6
3,1/31/2022,Xborder,Battambang,TH03542A,Big Red 50g/40g/35g,"""Colgate"" Toothpaste Fresh Cool Mint 40g.",216,77,1,0,...,77,0,0,0,0,0,77,0,0,77
4,1/31/2022,Xborder,Battambang,1524934,Big Red 50g/40g/35g,"""Colgate"" Toothpaste Double Cool Stripe 35g En...",216,77,50,0,...,3858,0,848,0,0,0,4706,531,0,4176


In [206]:
sub_df = sub_df[["Date", "Distributor", "PPGs", "Code", "DESCRIPTION PRODUCT",'SI (VOL)',  'SI FOC (VOL)', "SO (VOL)",'SO FOC (VOL)', "INV (VOL)"]]
sub_df

,Date,Distributor,PPGs,Code,DESCRIPTION PRODUCT,SI (VOL),SI FOC (VOL),SO (VOL),SO FOC (VOL),INV (VOL)
0,1/31/2022,Battambang,Big Red 25g/20g,1523871,"""Colgate"" Toothpaste Great Regular Flavor 20g.",0,0,0,,0
1,1/31/2022,Battambang,Big Red 50g/40g/35g,61007304,"""Colgate"" Toothpaste Great Regular Flavor 35g",0,0,0,,0
2,1/31/2022,Battambang,Big Red 50g/40g/35g,TH03541A,"""Colgate"" Toothpaste Great Regular Flavor 40g.",0,0,0,,0
3,1/31/2022,Battambang,Big Red 50g/40g/35g,TH03542A,"""Colgate"" Toothpaste Fresh Cool Mint 40g.",0,0,0,,1
4,1/31/2022,Battambang,Big Red 50g/40g/35g,1524934,"""Colgate"" Toothpaste Double Cool Stripe 35g En...",0,11,7,,54
...,...,...,...,...,...,...,...,...,...,...
42009,11/30/2022,Koh Kong,PTX SC 450/500ml (new pump),61001483,"""Protex"" Shower Cream Smooth Care 450ml",0,0,0,,0
42010,11/30/2022,Koh Kong,PTX SC 450/500ml (new pump),61001474,"""Protex"" Shower Cream Fresh 450ml",0,0,0,,0
42011,11/30/2022,Koh Kong,PTX SC 450/500ml (new pump),61021900,"""Protex"" Shower Cream Blossom Care 450ml",0,0,0,,0
42012,11/30/2022,Koh Kong,PTX SC 450/500ml (new pump),TH03480A,"Protex"" Shower Cream Men Charcoal- 450ml+50ml",0,0,0,,0


In [207]:
sub_df['Date'] = pd.to_datetime(sub_df['Date'])
sub_df['Date'] = sub_df['Date'].to_numpy().astype('datetime64[M]')
sub_df[['SI (VOL)', 'SI FOC (VOL)','SO (VOL)', 'SO FOC (VOL)', 'INV (VOL)']] = sub_df[['SI (VOL)', 'SI FOC (VOL)','SO (VOL)', 'SO FOC (VOL)', 'INV (VOL)']].apply(pd.to_numeric,downcast='float')
# sub_df['SI (VOL)']  = sub_df['SI (VOL)'] + sub_df['SI FOC (VOL)']
sub_df['SI (VOL)']  = sub_df['SI (VOL)']

In [208]:
sub_df = sub_df[["Date", "Distributor", "PPGs","Code", "DESCRIPTION PRODUCT", "SI (VOL)", "SO (VOL)", "INV (VOL)"]]

In [209]:
sub_df = sub_df.groupby(["Date", "Distributor", "PPGs","Code", "DESCRIPTION PRODUCT"], as_index=False).sum()

In [210]:
sub_df

,Date,Distributor,PPGs,Code,DESCRIPTION PRODUCT,SI (VOL),SO (VOL),INV (VOL)
0,2022-01-01,Banteay Meanchey,,61002888,"""Palmolive"" Naturals Ultra Smooth Shampoo & Co...",0.0,10.0,5.0
1,2022-01-01,Banteay Meanchey,,TH03447A,"""Palmolive""Shampoo& Conditioner Intensive Mois...",0.0,0.0,2.0
2,2022-01-01,Banteay Meanchey,,TH03448A,"""Palmolive""Shampoo& Conditioner Ultra Smooth G...",0.0,1.0,0.0
3,2022-01-01,Banteay Meanchey,,TH03449A,"""Palmolive""Shampoo& Conditioner Silky Straight...",0.0,1.0,0.0
4,2022-01-01,Banteay Meanchey,#N/A,1051531,"""Colgate"" Toothbrush Slim Soft Deep Clean",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
41650,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,61012801,"""Colgate"" Toothpaste Total Advanced Fresh 150g",0.0,0.0,0.0
41651,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,TH02939A,"""Colgate"" Toothpaste Total Advanced Fresh 150g",0.0,0.0,1.0
41652,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,TH02940A,"""Colgate"" Toothpaste Total Charcoal Deep Clean...",0.0,0.0,0.0
41653,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,TH02941A,"""Colgate"" Toothpaste Total Pro Breath Health 150g",0.0,0.0,0.0


In [ ]:
sheet_name = 'SO'
so_df = read_data(key_google_sheet, sheet_name)

so_df = so_df[["Date_time", "Customer_map", "PPGs","Inventory ID", "Description", "Total Quantity", "Order Type"]]
so_df.columns = ["Date_time", "Customer_map", "PPG", "SKU No.", "SKU Description", "Total Quantity", "Order Type"]
# so_df['Type'] = so_df['Order Type'].apply(lambda x: -1.0 if x == "CM" else 1.0)
so_df['Date_time']  =pd.to_datetime(so_df['Date_time'])
so_df['Total Quantity'] = so_df['Total Quantity'].str.replace(",", '')
so_df['Total Quantity'] = so_df['Total Quantity'].astype('float')
# so_df['Total Quantity'] = so_df['Total Quantity'] * so_df['Type']

# so_df = so_df.groupby(['Date_time', 'SKU No.'], as_index=False)[['Total Quantity']].sum()
# so_df.columns = ['Date', 'SKU No.', 'SO']


In [ ]:
so_df  = so_df[so_df['Customer_map'] != "#N/A"]
so_df = so_df[so_df['Customer_map'] != "Colgate (Claim)"]
so_df = so_df[so_df['Order Type'] != "CM"]

In [ ]:
so_df = so_df[["Date_time", "Customer_map", "PPG", "SKU No.", "SKU Description", "Total Quantity"]]
so_df.columns = ["Date_time", "Customer_map", "PPG","SKU No.", "SKU Description", "Total Quantity"]
so_df.columns = ['Date', 'Distributor', 'PPGs', 'SKU No.', 'SKU Description', 'SO (VOL)']

In [ ]:
so_df = so_df[['Date', 'Distributor',"PPGs" ,'SKU No.', 'SKU Description', 'SO (VOL)']]
so_df  = so_df.groupby(['Date', 'Distributor', 'PPGs','SKU No.', 'SKU Description'], as_index=False).sum()

In [ ]:
so_df

In [ ]:
sub_df

,Date,Distributor,PPGs,Code,DESCRIPTION PRODUCT,SI (VOL),SO (VOL),INV (VOL)
0,2022-01-01,Banteay Meanchey,,61002888,"""Palmolive"" Naturals Ultra Smooth Shampoo & Co...",0.0,10.0,5.0
1,2022-01-01,Banteay Meanchey,,TH03447A,"""Palmolive""Shampoo& Conditioner Intensive Mois...",0.0,0.0,2.0
2,2022-01-01,Banteay Meanchey,,TH03448A,"""Palmolive""Shampoo& Conditioner Ultra Smooth G...",0.0,1.0,0.0
3,2022-01-01,Banteay Meanchey,,TH03449A,"""Palmolive""Shampoo& Conditioner Silky Straight...",0.0,1.0,0.0
4,2022-01-01,Banteay Meanchey,#N/A,1051531,"""Colgate"" Toothbrush Slim Soft Deep Clean",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
41650,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,61012801,"""Colgate"" Toothpaste Total Advanced Fresh 150g",0.0,0.0,0.0
41651,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,TH02939A,"""Colgate"" Toothpaste Total Advanced Fresh 150g",0.0,0.0,1.0
41652,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,TH02940A,"""Colgate"" Toothpaste Total Charcoal Deep Clean...",0.0,0.0,0.0
41653,2022-11-01,Udor Meanchey,Total 110 / 150 g Single,TH02941A,"""Colgate"" Toothpaste Total Pro Breath Health 150g",0.0,0.0,0.0


In [ ]:
sub_df = sub_df[['Date', 'Distributor', 'PPGs', 'Code', 'DESCRIPTION PRODUCT', 'SI (VOL)']]
sub_df.columns = ['Date', 'Distributor', 'PPGs', 'SKU No.', 'SKU Description', 'SI (VOL)']

In [ ]:
res = sub_df.merge(so_df, on = ['Date', 'Distributor', 'PPGs', 'SKU No.'], how='outer')

In [ ]:
res = res[['Date', 'Distributor', 'PPGs'	, 'SKU No.', 'SKU Description_x','SI (VOL)','SO (VOL)']].sort_values('Date')

In [ ]:
res['SO (VOL)'] = res['SO (VOL)'].fillna(0)
res['SI (VOL)'] = res['SI (VOL)'].fillna(0)

In [ ]:
res['Diff'] =  res["SO (VOL)"] - res["SI (VOL)"]

In [ ]:
res

,Date,CPnA's customer,Distributor,PPGs,SKU No.,SKU Desc,SO (VOL) CPnA,SI (VOL)
0,2022-01-01,Banteay Meanchey,,61002888,"""Palmolive"" Naturals Ultra Smooth Shampoo & Co...",0.0,0.00,0.00
2558,2022-01-01,Pursat,COL MAX FRESH 160GM,61002917,Colgate Toothpaste Max Fresh Minty Blast 160g(...,0.0,0.00,0.00
2559,2022-01-01,Pursat,COL MAX FRESH 160GM,61002918,Colgate Toothpaste Max Fresh Minty Blast 160g,0.0,0.00,0.00
2560,2022-01-01,Pursat,COL MAX FRESH 160GM,61015275,"""Colgate""Toothpaste Max Fresh Peppermint Ice 150g",0.0,0.00,0.00
2561,2022-01-01,Pursat,COL MAX FRESH 160GM,61015886,"""Colgate""Toothpaste Max Fresh Peppermint Ice 1...",0.0,0.00,0.00
...,...,...,...,...,...,...,...,...
39173,2022-11-01,Kandal,Big Red 50g/40g/35g,61007304,"""Colgate"" Toothpaste Great Regular Flavor 35g",0.0,0.00,0.00
39172,2022-11-01,Kandal,Big Red 50g/40g/35g,1524934,"""Colgate"" Toothpaste Double Cool Stripe 35g En...",20.0,23.38,3.38
39171,2022-11-01,Kandal,Big Red 145/150/170/180g Twin,1520163,"""Colgate"" Toothpaste Double Cool Stripe 140g​​...",0.0,0.00,0.00
39177,2022-11-01,Kandal,CDC Kids 40g,61003154,"""Colgate"" Toothpaste Minions Fun Mint 40g.",0.0,0.00,0.00


In [ ]:
res.to_excel("demo.xlsx", index=False)

In [ ]:
res.columns  = ['Date', "CPnA's customer", "Distributor", 'PPGs',"SKU No.", "SKU Desc", "SO (VOL) CPnA", "SI (VOL)"]

In [ ]:
# res[["SO (VOL) CPnA", "SI (VOL)"]] = res[["SO (VOL) CPnA", "SI (VOL)"]].apply(np.ceil)

In [ ]:
test_df = res[res["Date"].notna()]
test_df[test_df['Diff'] <= 1 ].size / test_df.size *100

KeyError: ignored

In [ ]:
res[res['Date'].isna()]

,Date,CPnA's customer,Distributor,PPGs,SKU No.,SKU Desc,SO (VOL) CPnA,SI (VOL),Diff
0,NaT,NaN,Banteay Meanchey,,NaN,NaN,NaN,0.0,NaN
1,NaT,NaN,Banteay Meanchey,,NaN,NaN,NaN,0.0,NaN
2,NaT,NaN,Banteay Meanchey,,NaN,NaN,NaN,0.0,NaN
3,NaT,NaN,Banteay Meanchey,,NaN,NaN,NaN,0.0,NaN
4,NaT,NaN,Banteay Meanchey,#N/A,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...
41650,NaT,NaN,Udor Meanchey,Total 110 / 150 g Single,NaN,NaN,NaN,0.0,NaN
41651,NaT,NaN,Udor Meanchey,Total 110 / 150 g Single,NaN,NaN,NaN,0.0,NaN
41652,NaT,NaN,Udor Meanchey,Total 110 / 150 g Single,NaN,NaN,NaN,0.0,NaN
41653,NaT,NaN,Udor Meanchey,Total 110 / 150 g Single,NaN,NaN,NaN,0.0,NaN


In [ ]:
res[res['Date'].isna()]['Distributor'].value_counts()

Phnom Penh 2                    1983
Phnom Penh 1                    1865
Svay Rieng                      1742
Pursat                          1725
Takeo                           1724
Kampong Chhnang                 1720
Soung                           1710
Koh Kong                        1710
Kampot                          1705
RTK+MDK+KRT+STR                 1702
Kampong Cham                    1676
Kampong Som                     1657
Kandal                          1646
Kampong Thom                    1635
Battambang                      1610
Prey Veng                       1562
Banteay Meanchey                1528
Siem Reap                       1416
Kampong Speu                    1395
Udor Meanchey + Preah Vihear    1199
Prek Tameak                     1110
Poipet                          1047
Prey Veng                        344
Kampong Speu                     335
Udor Meanchey                    189
Banteay  Meanchey                186
Name: Distributor, dtype: int64

In [160]:
sheet_name = 'Database'
db_df = read_data(key_google_sheet, sheet_name)

In [161]:
db_df

,Date,Distributor,SKU No.,Category,SubCate,Subbrand,SKU Desc,INV (Vol),SI (Vol),SO (Vol),SO (Vol) _PP,SO (Vol) _PPP
0,1-Jan-2022,CP&A,1051110,Oral Care,Manual TB,Colgate Slim Soft Man TB,"""Colgate""Toothbrush Slim Soft Gentle Clean 1*3",439,500,264,,
1,1-Jan-2022,CP&A,1051118,Oral Care,Manual TB,Colgate Slim Soft Man TB,"""Colgate"" Toothbrush Slim Soft Gentle Clean",825,,233,,
2,1-Jan-2022,CP&A,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",78,,4,,
3,1-Jan-2022,CP&A,1051803,Oral Care,Manual TB,Colgate Classic Man TB,"""Colgate"" Toothbrush Deluxe Child",90,,46,,
4,1-Jan-2022,CP&A,1051826,Oral Care,Manual TB,Colgate Kids Man TB,"""Colgate"" Toothbrush Value Kids",198,50,7,,
...,...,...,...,...,...,...,...,...,...,...,...,...
42739,1-Mar-2022,Udor Meanchey + Preah Vihear,VN00608A,Oral Care,Manual TB,Colgate Extra Clean Man TB,"""Colgate"" Toothbrush Extra clean 3+2 with cap",0,0,0,0,0
42740,1-Mar-2022,Udor Meanchey + Preah Vihear,VN00758A,Oral Care,Manual TB,Colgate Premier Man TB,"""Colgate"" Toothbrush Gum Clean","1,650",100,73,22,32
42741,1-Mar-2022,Udor Meanchey + Preah Vihear,th01989A,Oral Care,Toothpaste,Colgate Kids TP,Colgate Toothpaste Bubble Fruit Flavor 40g,0,0,0,0,0
42742,1-Mar-2022,Udor Meanchey + Preah Vihear,th02855a,Personal Care,Body Wash,Palmolive Naturals BW,"""Palmolive"" Shower Gel Nourish Delight 500 ml",0,0,0,0,0


In [162]:
db_df.columns

Index(['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand',
       'SKU Desc', 'INV (Vol)', 'SI (Vol)', 'SO (Vol)', 'SO (Vol) _PP',
       'SO (Vol) _PPP'],
      dtype='object')

In [163]:
db_df  = db_df[['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand',
       'SKU Desc', 'INV (Vol)', 'SI (Vol)', 'SO (Vol)']]

In [164]:
db_df['Date'] =  pd.to_datetime(db_df['Date'])

<ipython-input-164-3c7555cf43cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_df['Date'] =  pd.to_datetime(db_df['Date'])


In [22]:
db_df_jan = db_df[db_df['Date'] == "2022-01-01"]
db_df_feb = db_df[db_df['Date'] == "2022-02-01"]
db_df_mar = db_df[db_df['Date'] == "2022-03-01"]
db_df_apr = db_df[db_df['Date'] == "2022-04-01"]

In [31]:
merged_df = db_df_mar.merge(db_df_feb, on =["Distributor", 'SKU No.', 'Category','SubCate', 'Subbrand', 'SKU Desc'], how = 'outer', suffixes = ('_3', '_2'))

In [36]:
db_df = merged_df[['Date_3', 'Distributor', "SKU No.", "Category", "SubCate", "Subbrand", "SKU Desc", "INV (Vol)_3", "SI (Vol)_3", 'SO (Vol)_3', "SO (Vol)_2"]]

In [39]:
merged_df = db_df.merge(db_df_jan, on =["Distributor", 'SKU No.', 'Category','SubCate', 'Subbrand', 'SKU Desc'], how = 'outer')

In [43]:
df_mar = merged_df[['Date_3', 'Distributor', "SKU No.", "Category", "SubCate", "Subbrand", "SKU Desc", "INV (Vol)_3", "SI (Vol)_3", 'SO (Vol)_3', "SO (Vol)_2", "SO (Vol)"]]

#  Create the DB for Dashboard 

In [183]:
sheet_name = 'Database'
db_df = read_data(key_google_sheet, sheet_name)
db_df.head()

,Date,Distributor,SKU No.,Category,SubCate,Subbrand,SKU Desc,INV (Vol),SI (Vol),SO (Vol),SO (Vol) _PP,SO (Vol) _PPP
0,1-Jan-2022,CP&A,1051110,Oral Care,Manual TB,Colgate Slim Soft Man TB,"""Colgate""Toothbrush Slim Soft Gentle Clean 1*3",439,500,264,,
1,1-Jan-2022,CP&A,1051118,Oral Care,Manual TB,Colgate Slim Soft Man TB,"""Colgate"" Toothbrush Slim Soft Gentle Clean",825,,233,,
2,1-Jan-2022,CP&A,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",78,,4,,
3,1-Jan-2022,CP&A,1051803,Oral Care,Manual TB,Colgate Classic Man TB,"""Colgate"" Toothbrush Deluxe Child",90,,46,,
4,1-Jan-2022,CP&A,1051826,Oral Care,Manual TB,Colgate Kids Man TB,"""Colgate"" Toothbrush Value Kids",198,50,7,,


In [184]:
db_df.columns

Index(['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand',
       'SKU Desc', 'INV (Vol)', 'SI (Vol)', 'SO (Vol)', 'SO (Vol) _PP',
       'SO (Vol) _PPP'],
      dtype='object')

In [185]:
db_df = db_df[['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand','SKU Desc', 'INV (Vol)', 'SI (Vol)', 'SO (Vol)']]

In [186]:
db_df['Date'] = pd.to_datetime(db_df['Date'])
db_df[['INV (Vol)', 'SI (Vol)', 'SO (Vol)']] = db_df[['INV (Vol)', 'SI (Vol)', 'SO (Vol)']].apply(lambda x: x.str.replace(",", ""))
db_df[['INV (Vol)', 'SI (Vol)', 'SO (Vol)']] = db_df[['INV (Vol)', 'SI (Vol)', 'SO (Vol)']].replace(r'^\s*$', np.nan, regex=True)
db_df[['INV (Vol)', 'SI (Vol)', 'SO (Vol)']] =db_df[['INV (Vol)', 'SI (Vol)', 'SO (Vol)']].astype('float')

In [187]:
df = db_df[['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand',
       'SKU Desc', 'INV (Vol)', 'SI (Vol)', 'SO (Vol)']]
df['Date'] = df['Date'].dt.to_period('M')
df['Date'] = df['Date'].dt.strftime("%Y-%m-01")
df


,Date,Distributor,SKU No.,Category,SubCate,Subbrand,SKU Desc,INV (Vol),SI (Vol),SO (Vol)
0,2022-01-01,CP&A,1051110,Oral Care,Manual TB,Colgate Slim Soft Man TB,"""Colgate""Toothbrush Slim Soft Gentle Clean 1*3",439.0,500.0,264.0
1,2022-01-01,CP&A,1051118,Oral Care,Manual TB,Colgate Slim Soft Man TB,"""Colgate"" Toothbrush Slim Soft Gentle Clean",825.0,NaN,233.0
2,2022-01-01,CP&A,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",78.0,NaN,4.0
3,2022-01-01,CP&A,1051803,Oral Care,Manual TB,Colgate Classic Man TB,"""Colgate"" Toothbrush Deluxe Child",90.0,NaN,46.0
4,2022-01-01,CP&A,1051826,Oral Care,Manual TB,Colgate Kids Man TB,"""Colgate"" Toothbrush Value Kids",198.0,50.0,7.0
...,...,...,...,...,...,...,...,...,...,...
42739,2022-03-01,Udor Meanchey + Preah Vihear,VN00608A,Oral Care,Manual TB,Colgate Extra Clean Man TB,"""Colgate"" Toothbrush Extra clean 3+2 with cap",0.0,0.0,0.0
42740,2022-03-01,Udor Meanchey + Preah Vihear,VN00758A,Oral Care,Manual TB,Colgate Premier Man TB,"""Colgate"" Toothbrush Gum Clean",1650.0,100.0,73.0
42741,2022-03-01,Udor Meanchey + Preah Vihear,th01989A,Oral Care,Toothpaste,Colgate Kids TP,Colgate Toothpaste Bubble Fruit Flavor 40g,0.0,0.0,0.0
42742,2022-03-01,Udor Meanchey + Preah Vihear,th02855a,Personal Care,Body Wash,Palmolive Naturals BW,"""Palmolive"" Shower Gel Nourish Delight 500 ml",0.0,0.0,0.0


In [188]:
df = df.sort_values(by=['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand',
       'SKU Desc']).groupby(['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand','SKU Desc'], as_index=False).sum()

In [189]:
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Distributor,SKU No.,Category,SubCate,Subbrand,SKU Desc,INV (Vol),SI (Vol),SO (Vol)
0,2022-01-01,Banteay Meanchey,1020708,#N/A,#N/A,#N/A,"""Palmolive""Shampoo Silky Straight Purple 6ml",0.0,0.0,0.0
1,2022-01-01,Banteay Meanchey,1050448,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Deluxe Adult",0.0,0.0,0.0
2,2022-01-01,Banteay Meanchey,1051110,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean Pack3",130.0,0.0,0.0
3,2022-01-01,Banteay Meanchey,1051118,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean",646.0,0.0,4.0
4,2022-01-01,Banteay Meanchey,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
42737,2022-11-01,Udor Meanchey,VN00605A,Oral Care,Manual TB,Colgate Extra Clean Man TB,"""Colgate"" Toothbrush Extra Clean Soft-Twinpack",0.0,0.0,0.0
42738,2022-11-01,Udor Meanchey,VN00758A,Oral Care,Manual TB,Colgate Premier Man TB,"""Colgate"" Toothbrush Gum Clean",403.0,0.0,2.0
42739,2022-11-01,Udor Meanchey,th01989A,Oral Care,Toothpaste,Colgate Kids TP,Colgate Toothpaste Bubble Fruit Flavor 40g,0.0,0.0,0.0
42740,2022-11-01,Udor Meanchey,th02855a,Personal Care,Body Wash,Palmolive Naturals BW,"""Palmolive"" Shower Gel Nourish Delight 500 ml",0.0,0.0,0.0


In [190]:
df_pp = df.copy()

In [191]:
df_pp['Date']=df_pp['Date']+pd.DateOffset(months=1)

In [192]:
df_pp

,Date,Distributor,SKU No.,Category,SubCate,Subbrand,SKU Desc,INV (Vol),SI (Vol),SO (Vol)
0,2022-02-01,Banteay Meanchey,1020708,#N/A,#N/A,#N/A,"""Palmolive""Shampoo Silky Straight Purple 6ml",0.0,0.0,0.0
1,2022-02-01,Banteay Meanchey,1050448,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Deluxe Adult",0.0,0.0,0.0
2,2022-02-01,Banteay Meanchey,1051110,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean Pack3",130.0,0.0,0.0
3,2022-02-01,Banteay Meanchey,1051118,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean",646.0,0.0,4.0
4,2022-02-01,Banteay Meanchey,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
42737,2022-12-01,Udor Meanchey,VN00605A,Oral Care,Manual TB,Colgate Extra Clean Man TB,"""Colgate"" Toothbrush Extra Clean Soft-Twinpack",0.0,0.0,0.0
42738,2022-12-01,Udor Meanchey,VN00758A,Oral Care,Manual TB,Colgate Premier Man TB,"""Colgate"" Toothbrush Gum Clean",403.0,0.0,2.0
42739,2022-12-01,Udor Meanchey,th01989A,Oral Care,Toothpaste,Colgate Kids TP,Colgate Toothpaste Bubble Fruit Flavor 40g,0.0,0.0,0.0
42740,2022-12-01,Udor Meanchey,th02855a,Personal Care,Body Wash,Palmolive Naturals BW,"""Palmolive"" Shower Gel Nourish Delight 500 ml",0.0,0.0,0.0


In [193]:
df_ppp = df.copy()
df_ppp['Date']=df_ppp['Date']+pd.DateOffset(months=2)

In [194]:
df_ppp

,Date,Distributor,SKU No.,Category,SubCate,Subbrand,SKU Desc,INV (Vol),SI (Vol),SO (Vol)
0,2022-03-01,Banteay Meanchey,1020708,#N/A,#N/A,#N/A,"""Palmolive""Shampoo Silky Straight Purple 6ml",0.0,0.0,0.0
1,2022-03-01,Banteay Meanchey,1050448,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Deluxe Adult",0.0,0.0,0.0
2,2022-03-01,Banteay Meanchey,1051110,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean Pack3",130.0,0.0,0.0
3,2022-03-01,Banteay Meanchey,1051118,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean",646.0,0.0,4.0
4,2022-03-01,Banteay Meanchey,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
42737,2023-01-01,Udor Meanchey,VN00605A,Oral Care,Manual TB,Colgate Extra Clean Man TB,"""Colgate"" Toothbrush Extra Clean Soft-Twinpack",0.0,0.0,0.0
42738,2023-01-01,Udor Meanchey,VN00758A,Oral Care,Manual TB,Colgate Premier Man TB,"""Colgate"" Toothbrush Gum Clean",403.0,0.0,2.0
42739,2023-01-01,Udor Meanchey,th01989A,Oral Care,Toothpaste,Colgate Kids TP,Colgate Toothpaste Bubble Fruit Flavor 40g,0.0,0.0,0.0
42740,2023-01-01,Udor Meanchey,th02855a,Personal Care,Body Wash,Palmolive Naturals BW,"""Palmolive"" Shower Gel Nourish Delight 500 ml",0.0,0.0,0.0


In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42742 entries, 0 to 42741
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         42742 non-null  datetime64[ns]
 1   Distributor  42742 non-null  object        
 2   SKU No.      42742 non-null  object        
 3   Category     42742 non-null  object        
 4   SubCate      42742 non-null  object        
 5   Subbrand     42742 non-null  object        
 6   SKU Desc     42742 non-null  object        
 7   INV (Vol)    42742 non-null  float64       
 8   SI (Vol)     42742 non-null  float64       
 9   SO (Vol)     42742 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 3.3+ MB


In [196]:
df_pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42742 entries, 0 to 42741
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         42742 non-null  datetime64[ns]
 1   Distributor  42742 non-null  object        
 2   SKU No.      42742 non-null  object        
 3   Category     42742 non-null  object        
 4   SubCate      42742 non-null  object        
 5   Subbrand     42742 non-null  object        
 6   SKU Desc     42742 non-null  object        
 7   INV (Vol)    42742 non-null  float64       
 8   SI (Vol)     42742 non-null  float64       
 9   SO (Vol)     42742 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 3.3+ MB


In [197]:
df   = df.merge(df_pp, on=["Date", "Distributor", "SKU No.","Category", "SubCate", "Subbrand", "SKU Desc"], suffixes= ('_cp', '_pp'), how= 'outer').merge(df_ppp, on=["Date", "Distributor", "SKU No.","Category", "SubCate", "Subbrand", "SKU Desc"] , how = 'outer')

In [198]:
df = df[['Date', 'Distributor', 'SKU No.', 'Category', 'SubCate', 'Subbrand','SKU Desc', "INV (Vol)_cp", "SI (Vol)_cp", "SO (Vol)_cp", "SO (Vol)_pp", "SO (Vol)"]]
df.columns = ['Date', 'Distributor', 'SKU_No', 'Category', 'SubCate', 'Subbrand','SKU_Desc', "INV_Vol", "SI_Vol", "SO_Vol", "SO_Vol_pp", "SO_Vol_ppp"]

In [199]:
df

,Date,Distributor,SKU_No,Category,SubCate,Subbrand,SKU_Desc,INV_Vol,SI_Vol,SO_Vol,SO_Vol_pp,SO_Vol_ppp
0,2022-01-01,Banteay Meanchey,1020708,#N/A,#N/A,#N/A,"""Palmolive""Shampoo Silky Straight Purple 6ml",0.0,0.0,0.0,NaN,NaN
1,2022-01-01,Banteay Meanchey,1050448,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Deluxe Adult",0.0,0.0,0.0,NaN,NaN
2,2022-01-01,Banteay Meanchey,1051110,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean Pack3",130.0,0.0,0.0,NaN,NaN
3,2022-01-01,Banteay Meanchey,1051118,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Gentle Clean",646.0,0.0,4.0,NaN,NaN
4,2022-01-01,Banteay Meanchey,1051531,#N/A,#N/A,#N/A,"""Colgate"" Toothbrush Slim Soft Deep Clean",0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
53741,2023-01-01,Udor Meanchey,VN00605A,Oral Care,Manual TB,Colgate Extra Clean Man TB,"""Colgate"" Toothbrush Extra Clean Soft-Twinpack",NaN,NaN,NaN,NaN,0.0
53742,2023-01-01,Udor Meanchey,VN00758A,Oral Care,Manual TB,Colgate Premier Man TB,"""Colgate"" Toothbrush Gum Clean",NaN,NaN,NaN,NaN,2.0
53743,2023-01-01,Udor Meanchey,th01989A,Oral Care,Toothpaste,Colgate Kids TP,Colgate Toothpaste Bubble Fruit Flavor 40g,NaN,NaN,NaN,NaN,0.0
53744,2023-01-01,Udor Meanchey,th02855a,Personal Care,Body Wash,Palmolive Naturals BW,"""Palmolive"" Shower Gel Nourish Delight 500 ml",NaN,NaN,NaN,NaN,0.0


In [182]:
#@title Load datafram to BigQuery
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
client = bigquery.Client(project ='cp-cdo-prod-ap-self-service')

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "cp-cdo-prod-ap-self-service.Thailand.TBL_CDT_INV_INDO"

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("Distributor", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("SKU_No", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Category", bigquery.enums.SqlTypeNames.STRING)
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 53746 rows and 12 columns to cp-cdo-prod-ap-self-service.Thailand.TBL_CDT_INV_INDO
